# 使用 gensim 訓練中文詞向量 (word2vec)

word2vec演算法通過將語詞(word)映射到N維的向量空間，然後基於這個詞向量可以進行聚類(Clustering)，找到近似詞以及詞性分析等相關的應用。

這篇文章裡將使用維基百科的中文語料，並使用python的gensim套件來訓練word2vec的模型。

![](https://imgur.com/S9na7ed.png)

### 下載相關訓練用的命令稿
下載[Alex-CHUN-YU/Word2vec](https://github.com/Alex-CHUN-YU/Word2vec)的Github專案

git clone https://github.com/Alex-CHUN-YU/Word2vec.git

### 下載中文維基數據
到 [中文維基數據dump](https://dumps.wikimedia.org/zhwiki/) 的目錄下找到最新的dump資料檔zhwiki-yyyymmdd-pages-articles.xml.bz2,

### 下載jieba字典檔
以"Download ZIP"的方式下載 [fxsjy/jieba](https://github.com/fxsjy/jieba), 解壓縮後將"extra_dict"整個目錄複製到"Word2vec/model"子目錄下

### 專案的檔案路徑佈局

```
Word2vec/
├──xxxx.ipynb 
├── main.py
├── segmentation.py
├── train.py
├── wiki_to_txt.py
├── stopwords.txt
├── model/
│   └──  extra_dict/
│       ├── dict.txt.big
│       ├── dict.txt.small
│       ├── idf.txt.big
│       └── stop_words.txt

data/
    └── zhwiki-20180220-pages-articles.xml.bz2
    ```

## 訓練流程
1. 取得中文維基數據，本次實驗是採用 2018/03/20 的資料。
2. 將下載後的維基數據置於與"data/"子目錄，再使用gensim.corpora的WikiCorpus函數來從wiki的xml檔案中提取出維基文章的語詞
3. 簡體轉繁體，再進行斷詞並同步過濾停用詞
4. 訓練並產生 word2vec 模型
5. 驗證word2vec近似詞以及詞性分析等相關功能

In [1]:
# switch off some warning messages
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import mmap
from tqdm import tqdm

import jieba
from gensim.corpora import WikiCorpus
from gensim.models import word2vec
from hanziconv import HanziConv
import matplotlib.pyplot as plt

In [2]:
# Project's root directory path
ROOT_DIR = os.getcwd()

# Training/verification data directory
DATA_PATH = "D:/Program/dataset/zhwiki-20180320-pages-articles/"

# Model data directory
MODEL_PATH = os.path.join(ROOT_DIR, "model")

# Set jieba Traditional Chinese dictionary file
JIEBA_DICTFILE_PATH = os.path.join(MODEL_PATH,"extra_dict", "dict.txt.big")

# Set traditional Chinese dictionary
jieba.set_dictionary(JIEBA_DICTFILE_PATH)

## 歩驟 1. 取得語料 (Corpus)
由於 word2vec 是基於非監督式學習，語料涵蓋的越全面，訓練出來的結果也會越漂亮。在本文中所採用的是維基百科於2018/03/20的dump檔，文章篇數共有 311571 篇。因為維基百科會定期更新備份資料，如果 8 月 20 號的備份不幸地被刪除了，也可以前往維基百科:資料庫下載挑選更近期的資料，不過請特別注意一點，我們要挑選的是以 pages-articles.xml.bz2 結尾的備份，而不是以 pages-articles-multistream.xml.bz2 結尾的備份，否則會在清理上出現一些異常，無法正常解析文章。

初始化WikiCorpus後，能藉由get_texts()可迭代每一篇wikipedia的文章，它所回傳的是一個tokens list，我們以空白符將這些 tokens 串接起來，統一輸出到同一份文字檔裡。這邊要注意一件事，get_texts()受wikicorpus.py中的變數ARTICLE_MIN_WORDS限制，只會回傳內容長度大於 50 的文章。

In [6]:
%%time

# Download the wiki data set and extract xml to plain txt
wiki_articles_xml_file = os.path.join(DATA_PATH, "zhwiki-20180320-pages-articles.xml.bz2")
wiki_articles_txt_file = os.path.join(DATA_PATH, "zhwiki_plaintext.txt")

# Use gensim.WikiCorpus to read corps of wiki XML
wiki_corpus = WikiCorpus(wiki_articles_xml_file, dictionary = {})

# Iteratively extracted words
with open(wiki_articles_txt_file, 'w', encoding='utf-8') as output:
    text_count = 0
    for text in wiki_corpus.get_texts():
        # Write words to the archive
        output.write(' '.join(text) + '\n')
        text_count += 1
        if text_count % 10000 == 0:
            print("目前已處理 %d 篇文章" % text_count)

print("轉檔完畢, 總共處理了 %d 篇文章!"% text_count)

目前已處理 10000 篇文章
目前已處理 20000 篇文章
目前已處理 30000 篇文章
目前已處理 40000 篇文章
目前已處理 50000 篇文章
目前已處理 60000 篇文章
目前已處理 70000 篇文章
目前已處理 80000 篇文章
目前已處理 90000 篇文章
目前已處理 100000 篇文章
目前已處理 110000 篇文章
目前已處理 120000 篇文章
目前已處理 130000 篇文章
目前已處理 140000 篇文章
目前已處理 150000 篇文章
目前已處理 160000 篇文章
目前已處理 170000 篇文章
目前已處理 180000 篇文章
目前已處理 190000 篇文章
目前已處理 200000 篇文章
目前已處理 210000 篇文章
目前已處理 220000 篇文章
目前已處理 230000 篇文章
目前已處理 240000 篇文章
目前已處理 250000 篇文章
目前已處理 260000 篇文章
目前已處理 270000 篇文章
目前已處理 280000 篇文章
目前已處理 290000 篇文章
目前已處理 300000 篇文章
目前已處理 310000 篇文章
轉檔完畢, 總共處理了 311571 篇文章!
Wall time: 45min 59s


## 歩驟 2. 進行中文斷詞與stop-word移除
我們有清完XML標籤的語料了，再來就是要把語料中每個句子，進一步拆解成語詞，這個步驟稱為「斷詞」。中文斷詞的工具有很多，這裏採用的是jieba。在wiki的中文文檔中有簡體跟繁體混在一起的情形，所以我們在斷詞前，還需加上一道繁簡轉換的手續。

In [7]:
# A function to get the number of rows of a text file
def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [8]:
# Simplified to Traditional
wiki_articles_zh_tw_file = os.path.join(DATA_PATH, "zhwiki_zh_tw.txt")

wiki_articles_zh_tw = open(wiki_articles_zh_tw_file, "w", encoding = "utf-8")

# Iterate into a plain text wiki document, and use HanziConv to convert to Traditional Chinese
with open(wiki_articles_txt_file, "r", encoding = "utf-8") as wiki_articles_txt:
    for line in tqdm(wiki_articles_txt, total=get_num_lines(wiki_articles_txt_file)):
        wiki_articles_zh_tw.write(HanziConv.toTraditional(line))
        
print("成功簡體轉繁體!")

wiki_articles_zh_tw.close()

100%|█████████████████████████████████████████████████████████████████████████| 311571/311571 [22:15<00:00, 233.29it/s]


成功簡體轉繁體!


In [9]:
# Chinese tokenization synchronization filtering stop words
stops_word_file = os.path.join(ROOT_DIR, "stopwords.txt")

stopwordset = set()

# Read stopword dictionary and save to stopwordset
with open("stopwords.txt", "r", encoding = "utf-8") as stopwords:
    for stopword in stopwords:
        stopwordset.add(stopword.strip('\n'))

# Reserve the result after segmented
wiki_articles_segmented_file = os.path.join(DATA_PATH, "zhwiki_segmented.txt")
wiki_articles_segmented = open(wiki_articles_segmented_file, "w", encoding = "utf-8")

# Iteratively transforming into traditional wiki documents and using jieba to segmented words
with open(wiki_articles_zh_tw_file, "r", encoding = "utf-8") as Corpus:
    for sentence in tqdm(Corpus, total=get_num_lines(wiki_articles_zh_tw_file)):
    #for sentence in Corpus:
        sentence = sentence.strip("\n")
        pos = jieba.cut(sentence, cut_all = False)
        for term in pos:
            if term not in stopwordset:
                wiki_articles_segmented.write(term + " ")
                
print("jieba 斷詞完畢，並已完成過濾停用詞!")
wiki_articles_zh_tw_file.close()

  0%|                                                                                       | 0/311571 [00:00<?, ?it/s]Building prefix dict from C:\Ipython\Keras_practice\3.7-word2vec-with-gensim\model\extra_dict\dict.txt.big ...
Dumping model to file cache C:\Users\user\AppData\Local\Temp\jieba.u9ef3b4ae54676b1e74b55fb64276c6b1.cache
Loading model cost 2.914 seconds.
Prefix dict has been built succesfully.
100%|█████████████████████████████████████████████████████████████████████████| 311571/311571 [46:27<00:00, 111.79it/s]


jieba 斷詞完畢，並已完成過濾停用詞!


AttributeError: 'str' object has no attribute 'close'

In [10]:
wiki_articles_zh_tw.close()

### 停用詞與Word2Vec
停用詞(stop word)就是像英文中的 the,a,this，中文的你我他，與其他詞相比顯得不怎麼重要，對文章主題也無關緊要的，就可以將它視為停用詞。而要排除停用詞的理由，其實與word2vec的實作有著相當大的關係。

在word2vec有一個概念叫"窗口( windows )"。

很顯然，一個詞的意涵跟他的左右鄰居很有關係，比如「雨越下越大，茶越充越淡」，什麼會「下」？「雨」會下，什麼會「淡」？茶會「淡」，這樣的類比舉不勝舉，那麼，若把思維逆轉過來呢？

顯然，我們或多或少能從左右鄰居是誰，猜出中間的是什麼，這很像我們國高中時天天在練的英文克漏字。那麼問題來了，左右鄰居有誰？能更精確地說，你要往左往右看幾個？假設我們以「孔乙己 一到 店 所有 喝酒 的 人 便都 看著 他 笑」為例，如果往左往右各看一個：
> 1 [孔乙己 一到] 店 所有 喝酒 的 人 便 都 看著 他 笑

> 2 [孔乙己 一到 店] 所有 喝酒 的 人 便 都 看著 他 笑

> 3 孔乙己 [一到 店 所有] 喝酒 的 人 便 都 看著 他 笑

> 4 孔乙己 一到 [店 所有 喝酒] 的 人 便 都 看著 他 笑

> 5 ......

這樣就構成了一個 size=1 的 windows，這個 1 是極端的例子，為了讓我們看看有停用詞跟沒停用詞差在哪，這句話去除了停用詞應該會變成：

> 1 孔乙己 一到 店 所有 喝酒 人 看著 笑

我們看看「人」的窗口變化，原本是「的 人 便」，後來是「喝酒 人 看著」，相比原本的情形，去除停用詞後，我們對「人」這個詞有更多認識，比如人會喝酒，人會看東西，當然啦，這是我以口語的表達，機器並不會這麼想，機器知道的是人跟喝酒會有某種關聯，跟看會有某種關聯，但儘管如此，也遠比本來的「的 人 便」好太多太多了。

## 歩驟 3. 訓練詞向量
這是最簡單的部分，同時也是最困難的部分，簡單的是程式碼，困難的是詞向量效能上的微調與後訓練。

相關參數:

* sentences:這是要訓練的句子集
* size:這表示的是訓練出的詞向量會有幾維
* alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
* sg:sg=1表示採用skip-gram,sg=0 表示採用cbow
* window:還記得孔乙己的例子嗎？能往左往右看幾個字的意思
* workers:執行緒數目，建議別超過 4
* min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象

In [ ]:
%%time

from gensim.models import word2vec

# 可參考 https://radimrehurek.com/gensim/models/word2vec.html 更多運用
print("word2vec模型訓練中...")

# Load file
sentence = word2vec.Text8Corpus(wiki_articles_segmented_file)

# Setting degree and Produce Model(Train)
model = word2vec.Word2Vec(sentence, size = 300, window = 10, min_count = 5, workers = 4, sg = 1)

# Save model
word2vec_model_file = os.path.join(MODEL_PATH, "zhwiki_word2vec.model")

# #保存模型，供日後使用
model.save("wiki_word2vec.model")
# model.wv.save_word2vec_format(word2vec_model_file, binary = True)

# #模型讀取方式
# # model = word2vec.Word2Vec.load("your_model_name")

print("word2vec模型已儲存完畢")

## 詞向量實驗
訓練完成後，讓我們來測試一下模型的效果。由於 gensim 會將整個模型讀了進來，所以記憶體會消耗相當多。

In [8]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec

word2vec_model_file = "D:/Program/model/wiki_word/wiki_word2vec.model"

word_vectors = word2vec.Word2Vec.load(word2vec_model_file)
# word_vectors = KeyedVectors.load_word2vec_format(word2vec_model_file)

In [25]:
print("詞彙相似詞前 5 排序")
query_list=['生病']
res = word_vectors.most_similar(query_list[0], topn = 5)
for item in res:
    print(item[0] + "," + str(item[1]))

詞彙相似詞前 5 排序
重病,0.6506224870681763
瞭病,0.6476729512214661
病倒,0.6445901393890381
病好,0.6436287760734558
患病,0.6367011070251465


In [26]:
print("計算兩個詞彙間 Cosine 相似度")
query_list=['妓女','嫖客']
res = word_vectors.similarity(query_list[0], query_list[1])
print(res)

計算兩個詞彙間 Cosine 相似度
0.712841532568


In [27]:
query_list=['妓女','嫖客','猛男']
print("%s之於%s，如%s之於" % (query_list[0], query_list[1], query_list[2]))
res = word_vectors.most_similar(positive = [query_list[0], query_list[1]], negative = [query_list[2]], topn = 5)
for item in res:
    print(item[0] + "," + str(item[1]))

妓女之於嫖客，如猛男之於
妓院,0.65537029504776
賣淫,0.5746345520019531
性工作者,0.5666040182113647
男妓,0.5513496398925781
老鴇,0.5243616104125977


# Reference:
* [以 gensim 訓練中文詞向量](http://zake7749.github.io/2016/08/28/word2vec-with-gensim/)
* [Alex-CHUN-YU/Word2vec](https://github.com/Alex-CHUN-YU/Word2vec)
* [word2vec-with-gensim](https://github.com/erhwenkuo/deep-learning-with-keras-notebooks/blob/master/8.4-word2vec-with-gensim.ipynb)
* [word2vec_tutorial](https://github.com/zake7749/word2vec_tutorial)